In [10]:
import numpy as np
from scipy import linalg
import scipy.sparse as sp

import sys
from pathlib import Path


negf_path = Path('..').resolve()
sys.path.insert(0, str(negf_path))


from device import Device
from hamiltonian import Hamiltonian

from typing import Tuple

def GzerozeroH_W_sparse(wmH: sp.spmatrix, t: sp.spmatrix) -> np.ndarray:
    """
    Surface Green's function calculation optimized for sparse matrices.
    """
    N = wmH.shape[0]
    

    wmH_dense = wmH.toarray()
    t_dense = t.toarray()

    A = np.zeros((2*N, 2*N), dtype=complex)
    B = np.zeros((2*N, 2*N), dtype=complex)
    
    A[:N, N:2*N] = np.eye(N)
    A[N:2*N, :N] = -t_dense.conj().T
    A[N:2*N, N:2*N] = wmH_dense
    
    B[:N, :N] = np.eye(N)
    B[N:2*N, N:2*N] = t_dense
    

    try:
        eigenvalues, eigenvectors = linalg.eig(A, B)
    except linalg.LinAlgError:
        print("Warning: Using pseudo-inverse for eigenvalue problem")
        B_pinv = linalg.pinv(B)
        eigenvalues, eigenvectors = linalg.eig(B_pinv @ A)
    

    magnitudes = np.abs(eigenvalues)

    valid_mask = np.isfinite(magnitudes) & (magnitudes > 1e-12)
    valid_eigenvalues = eigenvalues[valid_mask]
    valid_eigenvectors = eigenvectors[:, valid_mask]
    valid_magnitudes = magnitudes[valid_mask]
    
    if len(valid_eigenvalues) == 0:
        raise ValueError("No valid eigenvalues found")
    

    real_parts = np.real(valid_eigenvalues)
    sorted_indices = np.lexsort((valid_magnitudes, real_parts))
    
 
    sorted_eigenvectors = valid_eigenvectors[:, sorted_indices]
    
    Z11 = sorted_eigenvectors[:N, :N]
    Z21 = sorted_eigenvectors[N:2*N, :N]
    

    Z11_inv = linalg.pinv(Z11, rtol=1e-12)
    Gzeta = Z21 @ Z11_inv

    
    return Gzeta

def calculate_self_energy_nanowire(H00: sp.spmatrix, H01: sp.spmatrix, 
                                  E: float, eta: float = 1e-6) -> np.ndarray:
    """
    Calculate self-energy for silicon nanowire system.
    
    Parameters:
    -----------
    H00 : sp.spmatrix
        Unit cell Hamiltonian (sparse)
    H01 : sp.spmatrix
        Coupling between unit cells (sparse)
    E : float
        Energy
    eta : float
        Small imaginary part
        
    Returns:
    --------
    np.ndarray
        Self-energy matrix
    """
    N = H00.shape[0]
    
    wmH = E * sp.eye(N, dtype=complex) - H00 + 1j * eta * sp.eye(N, dtype=complex)
    
    print(f"Energy E = {E}")
    print(f"Matrix size N = {N}")
    print(f"wmH shape: {wmH.shape}")
    print(f"H01 shape: {H01.shape}")
    
    wmH_dense = wmH.toarray()
    H01_dense = H01.toarray()
    
    print(f"wmH trace: {np.trace(wmH_dense):.6f}")
    print(f"H01 trace: {np.trace(H01_dense):.6f}")
    print(f"wmH norm: {np.linalg.norm(wmH_dense):.6f}")
    print(f"H01 norm: {np.linalg.norm(H01_dense):.6f}")
    
    if np.allclose(wmH_dense, 0):
        print("WARNING: wmH is zero!")
        return np.zeros((N, N), dtype=complex)
    
    if np.allclose(H01_dense, 0):
        print("WARNING: H01 is zero!")
        return np.zeros((N, N), dtype=complex)
    
    Gzeta = GzerozeroH_W_sparse(wmH, H01)

    
    # Compute self-energy: Σ = H01 * Gζ * H01^†
    selfenergy = H01_dense @ Gzeta @ H01_dense.conj().T
    
    print(f"Self-energy trace: {np.trace(selfenergy):.6f}")
    print(f"Self-energy norm: {np.linalg.norm(selfenergy):.6f}")
    
    return selfenergy

def test_nanowire_self_energy():
    """
    Test function for your nanowire system.
    """
    print("Testing nanowire self-energy calculation...")
    
    dev = Device(channel_length=3e-9, channel_thickness=1.5e-9)
    ham = Hamiltonian(dev)
    
    # Get H00 and H01 for left lead
    H00, H01 = ham.get_H00_H01(0.5, side="left", sparse=True)
    
    print(f"H00 shape: {H00.shape}")
    print(f"H01 shape: {H01.shape}")
    print(f"H00 type: {type(H00)}")
    print(f"H01 type: {type(H01)}")
    
    # Test at different energies
    energies = [-2.0, -1.0, 0.0, 1.0, 100.0]
    
    for E in energies:
        print(f"\n{'='*60}")
        print(f"Testing at energy E = {E}")
        
        try:
            selfenergy = calculate_self_energy_nanowire(H00, H01, E)
            print(f"Self-energy calculation successful!")
            print(f"Self-energy shape: {selfenergy.shape}")
            
            if np.allclose(selfenergy, 0):
                print("WARNING: Self-energy is zero!")
            else:
                print("Self-energy is non-zero ✓")
                print(f"Self-energy max magnitude: {np.max(np.abs(selfenergy)):.6f}")
                
        except Exception as e:
            print(f"Error in self-energy calculation: {e}")
            import traceback
            traceback.print_exc()

def test_both_leads():

    
    dev = Device(channel_length=3e-9, channel_thickness=1.5e-9)
    ham = Hamiltonian(dev)
    

    H00_left, H01_left = ham.get_H00_H01(0.5, side="left", sparse=True)
    
    E = 0.0  # Test at Fermi level
    try:
        selfenergy_left = calculate_self_energy_nanowire(H00_left, H01_left, E)
        print(f"Left lead self-energy calculated successfully!")
    except Exception as e:
        print(f"Error in left lead: {e}")
    
    # Test right lead
    print("\n" + "="*60)
    print("RIGHT LEAD")
    H00_right, H01_right = ham.get_H00_H01(0.5, side="right", sparse=True)
    
    try:
        selfenergy_right = calculate_self_energy_nanowire(H00_right, H01_right, E)
        print(f"Right lead self-energy calculated successfully!")
    except Exception as e:
        print(f"Error in right lead: {e}")

if __name__ == "__main__":
    # Test the nanowire-specific implementation
    test_nanowire_self_energy()
    
    print("\n" + "="*60)
    
    # Test both leads
    test_both_leads()

Testing nanowire self-energy calculation...
H00 shape: (240, 240)
H01 shape: (240, 240)
H00 type: <class 'scipy.sparse._csc.csc_matrix'>
H01 type: <class 'scipy.sparse._csc.csc_matrix'>

Testing at energy E = -2.0
Energy E = -2.0
Matrix size N = 240
wmH shape: (240, 240)
H01 shape: (240, 240)
wmH trace: -2274.401680+0.000240j
H01 trace: 0.000000+0.000000j
wmH norm: 290.187084
H01 norm: 40.229066
Self-energy trace: -50.514286+0.000000j
Self-energy norm: 41.134137
Self-energy calculation successful!
Self-energy shape: (240, 240)
Self-energy is non-zero ✓
Self-energy max magnitude: 5.555218

Testing at energy E = -1.0
Energy E = -1.0
Matrix size N = 240
wmH shape: (240, 240)
H01 shape: (240, 240)
wmH trace: -2034.401680+0.000240j
H01 trace: 0.000000+0.000000j
wmH norm: 282.665421
H01 norm: 40.229066
Self-energy trace: -38.068379-0.000000j
Self-energy norm: 50.888072
Self-energy calculation successful!
Self-energy shape: (240, 240)
Self-energy is non-zero ✓
Self-energy max magnitude: 5.545

In [2]:
A = calculate_self_energy_simple(H00, H01, 2.0)

NameError: name 'calculate_self_energy_simple' is not defined

In [ ]:
A = spa.csc_matrix(A)

In [ ]:
A.nnz

0